In [13]:
# header
import pandas as pd
import csv
import AddIPAFlex
import MatchHeadwords
from GenerateLexDir import literal_eval_col
from IPython.display import display

# load in df's
flexicon = pd.read_csv('flexicon_3_3.csv', keep_default_na=False)
senses = pd.read_csv('SensesMERGE.csv', index_col='sense_id', keep_default_na=False)

# take things literally
#literal_eval_col(flexicon, 'sense')
literal_eval_col(flexicon, 'note')
literal_eval_col(flexicon, 'other_sources')
literal_eval_col(flexicon, 'these_vars')
literal_eval_col(flexicon, 'variant_of')

literal_eval_col(senses, 'gloss')
literal_eval_col(senses, 'def')

In [14]:
# first, check for perfect duplicate indices
for i in flexicon.index:
    if 1 < len(flexicon.loc[i]) < 10:
        display(flexicon.loc[i])

In [15]:
# now, check for duplicate guids
# create index using guid
guid = lambda s : s.split(sep='_')[-1][:8]
guids = [ guid(id) for id in flexicon['entry_id'] ]
flexicon.loc[:, 'guids'] = guids
flexicon = flexicon.set_index('guids')
flexicon.head()

,entry_id,bibliography,date,date_modified,headword,morph_type,note,other_sources,pronunciation,sense,these_vars,variant_of
guids,,,,,,,,,,,,
002ca005,wao taah_002ca005-1366-4a7b-aff1-ac10db10b768,Fieldnotes Epps/Obert; 2018,2019-01-14T03:46:55Z,,wao taah,phrase,None,None,waʔoʔ ta:h,['802366cc-fff6-4c35-bff4-a403f4b4130d'],None,{}
022d930d,janooh_022d930d-b619-4a77-8ff4-fa5ea30739e8,Fieldnotes Epps/Obert; 2018,2018-07-07T20:57:29Z,,janooh,stem,{},None,jaˈno:h,['9e409799-d500-44e1-97b4-66925497edde'],None,{}
022f0eb2,p’aa hẽnh_022f0eb2-2301-40e8-809c-927ccebeff3c,Fieldnotes Epps/Obert; 2018,2018-07-15T20:18:56Z,,p’aa hẽnh,phrase,None,{'Sil Dict 2011': 'pʹaa hẽnh'},paʔaʔ hẽɲ,['3954f8c6-283b-4e9f-a9b1-5e9ea6096fcc'],{'pʹaa hẽnh_988a43b7-7d72-4d87-be1d-c63c761e1d...,{}
024eb574,dajëp_024eb574-aa9a-4bea-adc6-12857406c334,Fieldnotes Epps/Obert; 2018,2018-07-10T00:09:23Z,,dajëp,stem,{},None,daˈjǝp˺,['a8675526-039d-459c-b4c8-c3434645200d'],None,{}
025c1d2a,tawarẽẽh1_025c1d2a-ce6b-4f4a-89c6-58f7995ce55a,Fieldnotes Epps/Obert; 2018,2018-03-08T07:17:53Z,,tawarẽẽh,stem,{},None,tawaˈɾẽ:ʔ,['edf56ebd-dec5-40c2-8444-09892cd2d358'],None,{}


In [16]:
# first, check for perfect duplicate indices
indices = []
for i in flexicon.index:
    if 1 < len(flexicon.loc[i]) < 10:
        indices.append(i)

with open('Duplicates_3_3.csv', 'w', newline='', encoding='utf8') as f:
    fieldnames = ['entry_id'] + [*flexicon.columns]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writerow({k:k for k in fieldnames})
    for i in indices:
        for index, row in flexicon.loc[i].iterrows():
            writer.writerow(dict( row ))
        writer.writerow({k: '' for k in fieldnames})

In [17]:
# done, let's load the data back in
resolved_duplicates = pd.read_csv('Duplicates_3_3-RESOLVED.csv', index_col='entry_id', keep_default_na=False)
print(resolved_duplicates.shape)
resolved_duplicates = resolved_duplicates[ [bool(x) for x in resolved_duplicates.index] ]
print(resolved_duplicates.shape)

(30, 11)
(16, 11)


In [18]:
# now let's drop the old indices and put in the replacements
flexicon.reset_index(drop=True)
flexicon.set_index('entry_id')
print(flexicon.shape)
for i in indices:
    if i in flexicon.index:
        flexicon = flexicon.drop(i)
print(flexicon.shape)
for index in resolved_duplicates.index:
    flexicon.loc[i] = resolved_duplicates.loc[i]
flexicon.shape

(1133, 12)
(1103, 12)


KeyError: 'b7f8608c'